In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

# Expectations

In [2]:
#Import POS file, AMT, Pallet Quantity file, Sku Conversion Sheet from THD to MAS 500

#Use AMT sheet, merge in pallet counts per sku. Assign this a new variable "OOS_Report".

#To "OOS Report" merge in POS OH units, fill na with 0. 

#Create a new column, labelled "OH PLTs". This column is the rounded of (OH Units / Pallet counts)

#Check for orders, import the MAS 500 data. Merge the sku conversion to the file.
#Change the customer number to store number. 

#Combine the open with the closed list. 

#Merge the sum of the totals by store by sku with the "OOS_Report" file. 

#Create boolean filter to take away anything that total doesn't equal 0 pallets.

# Live Code

In [3]:
POS_File = pd.read_excel(r'C:\Users\kawaljit_s\Documents\Kellogg\POS\Weekly POS\THD NEW POS Managers_20190513_094133.xlsx', sheet_name=6)
AMT = pd.read_excel(r'C:\Users\kawaljit_s\Documents\Kellogg\AMT Lists\2019 AMTv3.xlsx')
Sku_Info = pd.read_excel(r'C:\Users\kawaljit_s\Documents\Python\THD Sku Conversion & Pallet Quantities.xlsx')

In [4]:
POS_File.head(3)

,BYO,Market,Store,Item,Item Description,2019
0,1,1,105,1001100505,Planting Mx 1.5cf,29
1,1,1,105,1001100512,ANGS Flwr/Veg 2cf,82
2,1,1,105,1001100514,Raised Bed/Pot 2cf,141


In [5]:
AMT.head(3)

,BYO NBR,MKT NBR,STR NBR,Sku,Sku Description,AMT,STR NAME,SET NAME
0,1,1,105,623779,2 CU FT KELLOGG GROMULCH,N,DULUTH,Kellogg National with 40QT Potting Mix and wit...
1,1,1,105,623787,2 CU FT KELLOGG AMEND,N,DULUTH,Kellogg National with 40QT Potting Mix and wit...
2,1,1,105,623788,3 CU FT KELLOGG GARDEN SOIL,N,DULUTH,Kellogg National with 40QT Potting Mix and wit...


In [6]:
Sku_Info.head(3)

,Item,Description,ITM,ITM DESC,PLT
0,6850,KGO All Natural Garden Soil 2CF,1.001101e+09,2 CU FT KELLOGG ORGANIC GARDEN SOIL,48.0
1,6490,KGO Raised Bed & Potting Mix 2CF,1.001101e+09,2 CU FT KELLOGG ORG RAISED BED MIX,48.0
2,6830,All Natural Potting Mix 40QT,1.003840e+09,40 QT KELLOGG POTTING MIX,65.0


In [7]:
AMT.drop(columns = ['STR NAME', 'SET NAME'], inplace = True)

In [8]:
AMT = AMT[AMT['AMT'] == 'Y']

In [9]:
OOS_Report = AMT.merge(Sku_Info, how = 'left', left_on = 'Sku', right_on = 'ITM')
OOS_Report.drop(columns = ['Item', 'Description', 'ITM', 'ITM DESC'], inplace = True)

In [10]:
OOS_Report['PLT'] = OOS_Report['PLT'].astype('int')

In [11]:
OOS_Report.head()

,BYO NBR,MKT NBR,STR NBR,Sku,Sku Description,AMT,PLT
0,1,1,105,1001100505,1.5 CU FT KELLOGG PLANTING MIX,Y,60
1,1,1,105,1001100512,2 CU FT KELLOGG ORGANIC GARDEN SOIL,Y,48
2,1,1,105,1001100514,2 CU FT KELLOGG ORG RAISED BED MIX,Y,48
3,1,1,105,1003839523,40 QT KELLOGG POTTING MIX,Y,65
4,1,1,106,1001100505,1.5 CU FT KELLOGG PLANTING MIX,Y,60


In [12]:
OOS_Report = OOS_Report.merge(POS_File, how = 'left', left_on = ['STR NBR', 'Sku'], right_on = ['Store', 'Item'])

In [13]:
OOS_Report.drop(columns = ['BYO', 'Market', 'Store', 'Item', 'Item Description'], inplace = True)

In [14]:
OOS_Report = OOS_Report.fillna(0)

In [15]:
OOS_Report[2019] = OOS_Report[2019].astype('int')

In [16]:
OOS_Report.columns = ['BYO NBR', 'MKT NBR', 'STR NBR', 'Sku', 'Sku Description', 'AMT', 'PLT', 'OH Units']

In [17]:
OOS_Report.head()

,BYO NBR,MKT NBR,STR NBR,Sku,Sku Description,AMT,PLT,OH Units
0,1,1,105,1001100505,1.5 CU FT KELLOGG PLANTING MIX,Y,60,29
1,1,1,105,1001100512,2 CU FT KELLOGG ORGANIC GARDEN SOIL,Y,48,82
2,1,1,105,1001100514,2 CU FT KELLOGG ORG RAISED BED MIX,Y,48,141
3,1,1,105,1003839523,40 QT KELLOGG POTTING MIX,Y,65,123
4,1,1,106,1001100505,1.5 CU FT KELLOGG PLANTING MIX,Y,60,303


In [18]:
OOS_Report['OH PLTs'] = (OOS_Report['OH Units'] / OOS_Report['PLT']).round().astype('int')

In [19]:
OOS_Report.head()

,BYO NBR,MKT NBR,STR NBR,Sku,Sku Description,AMT,PLT,OH Units,OH PLTs
0,1,1,105,1001100505,1.5 CU FT KELLOGG PLANTING MIX,Y,60,29,0
1,1,1,105,1001100512,2 CU FT KELLOGG ORGANIC GARDEN SOIL,Y,48,82,2
2,1,1,105,1001100514,2 CU FT KELLOGG ORG RAISED BED MIX,Y,48,141,3
3,1,1,105,1003839523,40 QT KELLOGG POTTING MIX,Y,65,123,2
4,1,1,106,1001100505,1.5 CU FT KELLOGG PLANTING MIX,Y,60,303,5


Output 1: All OOS (Not counting on order)

In [21]:
True_OOS = OOS_Report[OOS_Report['OH PLTs'] == 0]

Continue: Adding In On-Orders

In [21]:
MAS500 = pd.read_excel(r'C:/Users/kawaljit_s/Documents/Python/Test Open & Closed File.xlsx', sheet_name=None)

In [22]:
MAS500 = MAS500['Open'].append(MAS500['Closed'], ignore_index=True, sort='True')

In [23]:
MAS500.drop(columns = ['Segment1', 'Segment1 Description', 'Segment2', 'Segment2 Description', 'Segment3', 'Segment3 Description'], inplace = True)

In [24]:
MAS500.head()

,Closed,Count Per Pallet,Create Date,Cust,Customer,Customer PO Number,Description,Ext Amt,Item,Pallet Ordered,Qty Ordered,SO Date,SO Nbr - Rel,Ship Date,Unit Price,Warehouse
0,NaT,70,2019-05-08,6004746,Home Depot #4746 - W. Richland,46980005,KGO Garden Soil Plus 1CF,2730.00,6770,13.0,910,2019-05-08,546150,2019-05-16,3.00,Longview
1,NaT,48,2019-05-02,6001409,Home Depot #1409 - HARRISON,9987666,KGO All Natural Garden Soil 2CF,4086.72,6850,18.0,864,2019-05-02,544891,2019-05-10,4.73,Garick Kansas
2,NaT,48,2019-05-08,6003016,Home Depot #3016 - KANSAS CITY,16970868,KGO All Natural Garden Soil 2CF,4086.72,6850,18.0,864,2019-05-08,546111,2019-05-16,4.73,Garick Kansas
3,NaT,65,2019-04-23,6004034,Home Depot #4034 - GRANTS PASS,34998529,All Natural Potting Mix 40QT,3912.35,6830,13.0,845,2019-04-23,543358,2019-05-01,4.63,Rexius
4,NaT,48,2019-05-02,6000164,Home Depot #164- PEACHTREE CITY,64970203,KGO All Natural Garden Soil 2CF,3737.28,6850,17.0,816,2019-05-02,545010,2019-05-10,4.58,Suwannee Lumber


In [25]:
MAS500_Totals = MAS500.groupby(by = ['Cust', 'Item']).sum()

In [26]:
MAS500_Totals = MAS500_Totals.drop_duplicates(keep = 'first')

In [27]:
MAS500_Totals.drop(columns = ['Count Per Pallet', 'Customer PO Number', 'Ext Amt', 'SO Nbr - Rel', 'Unit Price'], inplace = True)

In [28]:
MAS500_Totals.head()

Pallet Ordered  Qty Ordered
Cust    Item                             
6000105 6160             5.0          300
        6490             3.0          144
        6830             4.0          260
        6850             5.0          240
6000134 6490             9.0          432

In [29]:
MAS500_Totals.reset_index(inplace = True)

In [30]:
MAS500_Totals['Cust'] = [x - 6000000 for x in MAS500_Totals['Cust']]

In [31]:
MAS500_Totals.tail()

,Cust,Item,Pallet Ordered,Qty Ordered
1407,8988,685,2.0,70
1408,8988,3214,2.0,150
1409,8995,6490,4.0,192
1410,8995,6810,9.0,585
1411,8995,6850,5.0,240


In [32]:
MAS500_Totals = MAS500_Totals.merge(Sku_Info, how = 'left', on = 'Item')
MAS500_Totals.drop(columns = ['Description', 'ITM DESC', 'PLT'], inplace = True)

In [33]:
MAS500_Totals.dropna(how = 'any', inplace = True)

In [34]:
MAS500_Totals['ITM'] = MAS500_Totals['ITM'].astype('int')

In [35]:
MAS500_Totals.head()

,Cust,Item,Pallet Ordered,Qty Ordered,ITM
0,105,6160,5.0,300,1001100505
1,105,6490,3.0,144,1001100514
2,105,6830,4.0,260,1003839523
3,105,6850,5.0,240,1001100512
4,134,6490,9.0,432,1001100514


In [36]:
OOS_Report.head()

,BYO NBR,MKT NBR,STR NBR,Sku,Sku Description,AMT,PLT,OH Units,OH PLTs
0,1,1,105,1001100505,1.5 CU FT KELLOGG PLANTING MIX,Y,60,45,1
1,1,1,105,1001100512,2 CU FT KELLOGG ORGANIC GARDEN SOIL,Y,48,147,3
2,1,1,105,1001100514,2 CU FT KELLOGG ORG RAISED BED MIX,Y,48,159,3
3,1,1,105,1003839523,40 QT KELLOGG POTTING MIX,Y,65,134,2
4,1,1,106,1001100505,1.5 CU FT KELLOGG PLANTING MIX,Y,60,303,5


In [37]:
POS_With_OO = OOS_Report.merge(MAS500_Totals, how = 'left', left_on = ['STR NBR', 'Sku'], right_on = ['Cust', 'ITM']).fillna(0)
POS_With_OO.head()

,BYO NBR,MKT NBR,STR NBR,Sku,Sku Description,AMT,PLT,OH Units,OH PLTs,Cust,Item,Pallet Ordered,Qty Ordered,ITM
0,1,1,105,1001100505,1.5 CU FT KELLOGG PLANTING MIX,Y,60,45,1,105.0,6160.0,5.0,300.0,1.001101e+09
1,1,1,105,1001100512,2 CU FT KELLOGG ORGANIC GARDEN SOIL,Y,48,147,3,105.0,6850.0,5.0,240.0,1.001101e+09
2,1,1,105,1001100514,2 CU FT KELLOGG ORG RAISED BED MIX,Y,48,159,3,105.0,6490.0,3.0,144.0,1.001101e+09
3,1,1,105,1003839523,40 QT KELLOGG POTTING MIX,Y,65,134,2,105.0,6830.0,4.0,260.0,1.003840e+09
4,1,1,106,1001100505,1.5 CU FT KELLOGG PLANTING MIX,Y,60,303,5,0.0,0.0,0.0,0.0,0.000000e+00


In [38]:
POS_With_OO.drop(columns = ['Cust', 'Item', 'ITM'], inplace = True)

In [39]:
POS_With_OO['OH & OO PLT'] = POS_With_OO['OH PLTs'] + POS_With_OO['Pallet Ordered']

In [40]:
POS_With_OO.head()

,BYO NBR,MKT NBR,STR NBR,Sku,Sku Description,AMT,PLT,OH Units,OH PLTs,Pallet Ordered,Qty Ordered,OH & OO PLT
0,1,1,105,1001100505,1.5 CU FT KELLOGG PLANTING MIX,Y,60,45,1,5.0,300.0,6.0
1,1,1,105,1001100512,2 CU FT KELLOGG ORGANIC GARDEN SOIL,Y,48,147,3,5.0,240.0,8.0
2,1,1,105,1001100514,2 CU FT KELLOGG ORG RAISED BED MIX,Y,48,159,3,3.0,144.0,6.0
3,1,1,105,1003839523,40 QT KELLOGG POTTING MIX,Y,65,134,2,4.0,260.0,6.0
4,1,1,106,1001100505,1.5 CU FT KELLOGG PLANTING MIX,Y,60,303,5,0.0,0.0,5.0


Output 2: OOS without an Order

In [42]:
Actual_OO = POS_With_OO[POS_With_OO['OH & OO PLT'] == 0]

Exporting Results

In [22]:
True_OOS.to_excel('All OOS skus.xlsx', index = False)
#Actual_OO.to_excel('OOS without Orders.xlsx', index = False)

# Plotting The Results